In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from datetime import datetime
import random
import os
import sys
# Get the current directory.
current_dir = os.getcwd()
# Add the `src` folder to the PYTHONPATH.
sys.path.append(os.path.join(current_dir, "../src"))
from scraping_functions import *

In [7]:
tournaments_df = pd.read_csv('../Data/tournaments.csv').set_index(['Name', 'Year'])
players_df = pd.read_csv('../Data/players.csv').set_index('Name')

matches_df = pd.read_csv('../Data/matches.csv')
matches_df = matches_df.drop(matches_df.columns[[0]], axis = 1)

In [8]:
matches_df.isna().sum()

Player 1                       0
Player 2                       0
Winner                         0
Score                          0
Tournament Name                0
Year                           0
Player 1 Ranking               0
Player 2 Ranking               0
Date                           0
Player 1 H2H Win Percentage    0
Surface                        0
dtype: int64

In [108]:
def find_ranking1(row, tournaments_df, players_df, player1 = True):
    name = row['Player 1'] if player1 else row['Player 2']
    tournament, year = row['Tournament Name'], row['Year']
    tournament_id = tournaments_df.loc[(tournament, year), 'Id'][0]
    try:
        url = 'https://www.atptour.com/en/scores/archive/%s/%d/%d/results' % (tournament.lower(), tournament_id, year)
        page = requests.get(url).text
    except Exception as e:
        return -1
    soup = BeautifulSoup(page, features="lxml")

    start_date = tournaments_df.loc[(tournament, year), 'Start Date'][0]
    try:
        name, player_id = name.lower().replace(" ", '-'), players_df.loc[name, 'Id']
        if 'jr.' in name:
            name = name[:-1]
    except Exception as e:
        print(e)
        return -1
    ranking_page = 'https://www.atptour.com/en/players/%s/%s/rankings-history' % (name, player_id)
    historical_ranking = requests.get(ranking_page).text
    if 'Bad Request' in historical_ranking:
        ranking_page = 'https://www.atptour.com/en/players/%s/%s/rankings-history' % (name, player_id[1])
        historical_ranking = requests.get(ranking_page).text
    print(ranking_page)
    soup1 = BeautifulSoup(historical_ranking, features="lxml")

    dates, contents, format = [], [], '%Y.%m.%d'
    date_to_ranking = {}
    for tag in soup1.find_all('td'):
        tag = str(tag)[4:]
        content = tag[:tag.find('<')].strip()
        contents.append(content)
    for index, content in enumerate(contents):
        try:
            date = datetime.strptime(content, format)
            dates.append(date)
            r = contents[index + 1]
            r = r[:-1] if 'T' in r else r
            ranking = int(r)
            date_to_ranking[content] = ranking
        except:
            continue
    ranking_date = -1
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if start_date < datetime.strptime(min(date_to_ranking.keys()), format):
        return 1000
    for index, date in enumerate(dates):
        if date <= start_date:
            ranking_date = date
            break
    try:
        ranking = date_to_ranking[datetime.strftime(ranking_date, format)]
    except:
        return -1
    return ranking

In [109]:
for index, row in matches_df.iterrows():
    if row['Player 1 Ranking'] == -1:
        val = find_ranking1(row, tournaments_df, players_df, player1=True)
        if val == -1:
            if (row['Tournament Name'] == 'Indian Wells') and (row['Year'] == 2023):
                matches_df.at[index, 'Player 1 Ranking'] = 1500
            else:
                print("problem")
                break
        else:      
            matches_df.at[index, 'Player 1 Ranking'] = val
        print(index)
    if row['Player 2 Ranking'] == -1:
        val = find_ranking1(row, tournaments_df, players_df, player1=False)
        if val == -1:
            if row['Tournament Name'] == 'Indian Wells' and row['Year'] == 2023:
                matches_df.at[index, 'Player 2 Ranking'] = 1500
            else:
                print("problem")
                break
        else:    
            matches_df.at[index, 'Player 2 Ranking'] = val
        print(index)

https://www.atptour.com/en/players/adam-walton/W09E/rankings-history
85669
https://www.atptour.com/en/players/alex-bogomolov-jr/B842/rankings-history
85964
https://www.atptour.com/en/players/alex-bogomolov-jr/B842/rankings-history
85978
https://www.atptour.com/en/players/alex-bogomolov-jr/B842/rankings-history
85985
https://www.atptour.com/en/players/luka-belic/BD05/rankings-history
86067
https://www.atptour.com/en/players/alex-bogomolov-jr/B842/rankings-history
86071
https://www.atptour.com/en/players/deni-zmak/Z262/rankings-history
86118
https://www.atptour.com/en/players/ante-pavic/PC78/rankings-history
86120
https://www.atptour.com/en/players/ante-pavic/PC78/rankings-history
86127
https://www.atptour.com/en/players/vilim-visak/V467/rankings-history
86128
https://www.atptour.com/en/players/antonio-sancic/SI06/rankings-history
86133
https://www.atptour.com/en/players/josko-topic/T762/rankings-history
86134
https://www.atptour.com/en/players/nikola-martinovic/MC66/rankings-history
861

In [107]:
players_df.loc['Adam Walton']

,Id
Name,
Adam Walton,sr:c
Adam Walton,W09E


In [110]:
matches_df[(matches_df['Player 1 Ranking'] == -1) | (matches_df['Player 2 Ranking'] == -1)]

,Player 1,Player 2,Winner,Score,Tournament Name,Year,Player 1 Ranking,Player 2 Ranking,Date,Player 1 H2H Win Percentage,Surface


In [111]:
matches_df.to_csv('matches.csv')